In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # ETL Process for Insurance Data
# MAGIC This notebook performs an ETL process on insurance data using PySpark. It loads data from Unity Catalog tables, processes it, and writes the results to a Delta table.

# COMMAND ----------

import logging
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, DateType, IntegerType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

try:
    # Load data from Unity Catalog tables
    logger.info("Loading data from Unity Catalog tables...")
    policy_df = spark.table("catalog.source_db.policydb")
    claims_df = spark.table("catalog.source_db.claimsdb")
    demographics_df = spark.table("catalog.source_db.demographicsdb")
    scores_df = spark.table("catalog.source_db.scores")
    aiml_insights_df = spark.table("catalog.source_db.aiml_insights")

# COMMAND ----------

    # Select relevant fields and cast types
    logger.info("Selecting relevant fields and casting types...")
    policy_df = policy_df.select(
        "policy_id", "customer_id", "policy_type", "policy_status",
        F.col("policy_start_date").cast(DateType()), 
        F.col("policy_end_date").cast(DateType()), 
        F.col("policy_term").cast(IntegerType()), 
        F.col("policy_premium").cast(DoubleType()), 
        F.col("total_premium_paid").cast(DoubleType()), 
        "renewal_status", "policy_addons"
    )

    claims_df = claims_df.select(
        "Claim_ID", "Policy_ID", 
        F.col("Claim_Date").cast(DateType()), 
        "Claim_Type", "Claim_Status", 
        F.col("Claim_Amount").cast(DoubleType()), 
        F.col("Claim_Payout").cast(DoubleType())
    )

    demographics_df = demographics_df.select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", 
        "City", "State", "Postal_Code", 
        F.col("Date_of_Birth").cast(DateType()), 
        "Gender", "Marital_Status", "Occupation", "Income_Level", "Customer_Segment"
    )

# COMMAND ----------

    # Join datasets
    logger.info("Joining datasets...")
    joined_df = demographics_df.join(policy_df, demographics_df.Customer_ID == policy_df.customer_id, "inner")
    joined_df = joined_df.join(claims_df, joined_df.policy_id == claims_df.Policy_ID, "inner")

# COMMAND ----------

    # Aggregate data
    logger.info("Aggregating data...")
    aggregated_df = joined_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.count("policy_id").alias("Policy_Count"),
        F.max("Claim_Date").alias("Recent_Claim_Date"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

    # Join with scores and AIML insights
    logger.info("Joining with scores and AIML insights...")
    final_df = aggregated_df.join(scores_df, "Customer_ID", "inner")
    final_df = final_df.join(aiml_insights_df, "Customer_ID", "inner")

# COMMAND ----------

    # Calculate additional fields
    logger.info("Calculating additional fields...")
    final_df = final_df.withColumn("Age", F.datediff(F.current_date(), F.col("Date_of_Birth")) / 365)
    final_df = final_df.withColumn("Claim_To_Premium_Ratio", F.when(F.col("total_premium_paid") != 0, F.col("Claim_Amount") / F.col("total_premium_paid")).otherwise(0))
    final_df = final_df.withColumn("Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0))
    final_df = final_df.withColumn("Retention_Rate", F.lit(0.85))
    final_df = final_df.withColumn("Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on"))
    final_df = final_df.withColumn("Upsell_Potential", F.lit("Premium Vehicle Coverage"))

# COMMAND ----------

    # Write the final DataFrame to a Delta table
    logger.info("Writing the final DataFrame to a Delta table...")
    final_df.write.format("delta").mode("overwrite").saveAsTable("catalog.target_db.customer_360")

    logger.info("ETL process completed successfully.")

except Exception as e:
    logger.error("An error occurred during the ETL process", exc_info=True)
